In [34]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

### How to

[sparkbyexamples](https://sparkbyexamples.com/spark/spark-read-csv-file-into-dataframe/)

In [11]:
spark = SparkSession.builder.appName("Bike_Rental").getOrCreate()

In [12]:
tripdata = spark.read.format("csv").options(header='true', inferSchema='true').load("201508_trip_data.csv")

stationdata = spark.read.format("csv").option("header", "true").load("201508_station_data.csv")

In [6]:
print(tripdata.first())

NameError: name 'tripdata' is not defined

In [32]:
print(tripdata.printSchema())

root
 |-- Trip ID: integer (nullable = true)
 |-- Duration: integer (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- Start Station: string (nullable = true)
 |-- Start Terminal: integer (nullable = true)
 |-- End Date: string (nullable = true)
 |-- End Station: string (nullable = true)
 |-- End Terminal: integer (nullable = true)
 |-- Bike #: integer (nullable = true)
 |-- Subscriber Type: string (nullable = true)
 |-- Zip Code: string (nullable = true)

None


In [34]:
print(stationdata.first())

Row(station_id='2', name='San Jose Diridon Caltrain Station', lat='37.329732', long='-121.901782', dockcount='27', landmark='San Jose', installation='8/6/2013')


In [39]:
print(stationdata.printSchema())

root
 |-- station_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- dockcount: string (nullable = true)
 |-- landmark: string (nullable = true)
 |-- installation: string (nullable = true)

None


In [38]:
#1 Available bikes
tripdata.select("Bike #").distinct().count()

668

In [41]:
#2 How many rental sites exist

stationdata.select("station_id").distinct().count()

70

In [54]:
#2.2 how many station exists in san francisco
stationdata.select("*").where("landmark == 'San Francisco'").distinct().count()

35

In [96]:
#3 which station has the most number of spots (docks) for bikes?
stationdata = stationdata.withColumn("dockcount", stationdata["dockcount"].cast('float'))



stationdata.select("*").orderBy("dockcount", ascending=False).first()

#stationdata.groupby().max("dockcount").show()

Row(station_id='2', name='San Jose Diridon Caltrain Station', lat='37.329732', long='-121.901782', dockcount=27.0, landmark='San Jose', installation='8/6/2013')

In [13]:
#4 How long is the average duration of a trip
tripdata.groupby().mean("Duration").show()

+------------------+
|     avg(Duration)|
+------------------+
|1046.0326611172604|
+------------------+



In [27]:
#5 How many bike trips tool place 


#overall = tripdata.select("*").distinct().count()
#per_starting_station = tripdata.select("*").groupby("Start Station").count().show()
#print("Overall = {}".format(overall))
per_bike = tripdata.select("*").orderBy("Bike #", 
                                        ascending=True).groupby("Bike #").count().show()


+------+-----+
|Bike #|count|
+------+-----+
|     9|  144|
|    10|  133|
|    11|   83|
|    12|   80|
|    13|  112|
|    14|  106|
|    15|   76|
|    16| 1026|
|    17|  153|
|    18|  133|
|    19|  172|
|    20|  113|
|    21|   94|
|    22|  160|
|    23|  143|
|    24|   28|
|    25|  175|
|    26|   39|
|    27|   82|
|    28|  166|
+------+-----+
only showing top 20 rows



In [42]:
#6 What is the percentage of rountrips

all_trips = tripdata.select("Trip ID").distinct().count()
trips_with_end = tripdata.select("*").where(F.col("End Station").isNotNull()).count()

print("Percentage of rount trips = {}".format(trips_with_end/all_trips*100))

Percentage of rount trips = 100.0


In [48]:
# on which day occurred the most bike trips

#Overall
tripdata.select("*").orderBy("End Date", ascending=True).



+-------+--------+-------------+--------------------+--------------+-------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|   Start Date|       Start Station|Start Terminal|     End Date|         End Station|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+-------------+--------------------+--------------+-------------+--------------------+------------+------+---------------+--------+
| 444322|     743|9/9/2014 9:46|Harry Bridges Pla...|            50|9/9/2014 9:59|San Francisco Cal...|          69|   538|     Subscriber|   94591|
| 444329|     548|9/9/2014 9:49|     2nd at Townsend|            61|9/9/2014 9:58|       5th at Howard|          57|   430|     Subscriber|   94107|
| 444338|     428|9/9/2014 9:50|San Francisco Cit...|            58|9/9/2014 9:58|  Powell Street BART|          39|   878|     Subscriber|   94102|
| 444336|     261|9/9/2014 9:53|Harry Bridges Pla...|            50|9/9/2014 9:57|Mechanics Plaza (...|   